# Day Ahead Market

In [1]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import requests
API_KEY = "3da11da4-cb1c-41ea-8f2e-e80e188b9e4b"
periodStart = "202509010000"
periodEnd = "202509300000"

import requests

url = ("https://web-api.tp.entsoe.eu/api?"
"documentType=A44&"
"processType=A16&"
"out_Domain=10YHU-MAVIR----U&"
"in_Domain=10YHU-MAVIR----U&"
"contract_MarketAgreement.type=A01&"
f"periodStart={periodStart}&"
f"periodEnd={periodEnd}&"
f"securityToken={API_KEY}")

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)


<?xml version="1.0" encoding="utf-8"?>
  <Publication_MarketDocument xmlns="urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3">
    <mRID>d6c14f9c2cea473494634d78c262877d</mRID>
    <revisionNumber>1</revisionNumber>
    <type>A44</type>
    <sender_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</sender_MarketParticipant.mRID>
    <sender_MarketParticipant.marketRole.type>A32</sender_MarketParticipant.marketRole.type>
    <receiver_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</receiver_MarketParticipant.mRID>
    <receiver_MarketParticipant.marketRole.type>A33</receiver_MarketParticipant.marketRole.type>
    <createdDateTime>2025-11-12T10:05:41Z</createdDateTime>
    <period.timeInterval>
      <start>2025-08-31T22:00Z</start>
      <end>2025-09-30T22:00Z</end>
    </period.timeInterval>
      <TimeSeries>
        <mRID>1</mRID>
        <auction.type>A01</auction.type>
        <businessType>A62</businessType>
        <in_Domain.mRID codingScheme="A01">10

In [3]:
# Initialize zf and xml_files from `response` whether ZIP or plain XML
import io, zipfile
from pathlib import Path


content = response.content
try:
    # Try to open as ZIP
    zf = zipfile.ZipFile(io.BytesIO(content))
    xml_files = zf.namelist()
except zipfile.BadZipFile:
    # Wrap plain XML into a single-file ZIP so downstream code works unchanged
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
        zout.writestr('response.xml', content)
    buf.seek(0)
    zf = zipfile.ZipFile(buf, 'r')
    xml_files = zf.namelist()

print(f"Ready: {len(xml_files)} XML file(s)")


Ready: 1 XML file(s)


In [4]:
# Extract XML files into a single DataFrame (store raw XML text)
import re


rows = []

for xml_file in xml_files:
    print(f"\nProcessing {xml_file}...")

    # Read XML content and decode (handle BOM safely)
    xml_bytes = zf.read(xml_file)
    xml_text = xml_bytes.decode('utf-8-sig', errors='replace')

    # Split concatenated XML documents while preserving the declaration
    docs = [d for d in re.split(r'(?=<\?xml\s)', xml_text) if d.strip()]
    if not docs:
        docs = [xml_text]

    for i, doc in enumerate(docs):
        # Ensure declaration at start; if missing, add a minimal one
        cleaned = doc.lstrip()
        if not cleaned.startswith('<?xml'):
            cleaned = '<?xml version="1.0" encoding="UTF-8"?>\n' + cleaned

        name = f"{xml_file}_part_{i}" if len(docs) > 1 else xml_file
        rows.append({'file': name, 'xml': cleaned})

xml_docs_df = pd.DataFrame(rows)
print(f"Collected {len(xml_docs_df)} XML documents into xml_docs_df")
display(xml_docs_df.head())



Processing response.xml...
Collected 1 XML documents into xml_docs_df


,file,xml
0,response.xml,"<?xml version=""1.0"" encoding=""utf-8""?>\n <Pub..."


In [5]:
# Parse a single balancing XML into one DataFrame (metadata + all points)
from lxml import etree as LET
from pathlib import Path
import pandas as pd


def _lname(tag: str) -> str:
    return tag.split('}')[-1] if '}' in tag else tag


def parse_balancing_xml_to_df(xml_text: str) -> pd.DataFrame:
    root = LET.fromstring(xml_text.encode('utf-8'))

    # Document-level metadata (leaf nodes outside Point)
    meta = {}
    for el in root.iter():
        ln = _lname(el.tag)
        if ln == 'Point':
            continue
        # skip if any Point descendant
        if el.xpath('.//*[local-name()="Point"]'):
            continue
        if len(list(el)) == 0:
            val = (el.text or '').strip()
            if val:
                key = f"doc.{ln}"
                # keep first occurrence
                if key not in meta:
                    meta[key] = val
    # include root attributes
    for k, v in root.attrib.items():
        meta[f"doc.@{k}"] = v

    rows = []
    # Iterate TimeSeries → Period → Point
    for ts in root.xpath('.//*[local-name()="TimeSeries"]'):
        ts_meta = {}
        for child in ts:
            ln = _lname(child.tag)
            if ln in ('Period', 'timeInterval'):
                continue
            # capture simple leaf text under TS (one level)
            if len(list(child)) == 0:
                ts_meta[f"ts.{ln}"] = (child.text or '').strip()
        for per in ts.xpath('.//*[local-name()="Period"]'):
            # period context
            start = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="start"]/text()')
            end = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="end"]/text()')
            res = per.xpath('.//*[local-name()="resolution"]/text()')
            per_meta = {
                'per.start': start[0] if start else None,
                'per.end': end[0] if end else None,
                'per.resolution': res[0] if res else None,
            }
            for pt in per.xpath('.//*[local-name()="Point"]'):
                row = {}
                # Point fields (flatten one level deep)
                for ch in list(pt):
                    ln = _lname(ch.tag)
                    if len(list(ch)) == 0:
                        row[f"pt.{ln}"] = (ch.text or '').strip()
                    else:
                        # flatten nested elements under Point one level
                        for sub in ch:
                            row[f"pt.{ln}.{_lname(sub.tag)}"] = (sub.text or '').strip()
                # Merge: document meta + TS meta + period meta + point
                full = {**meta, **ts_meta, **per_meta, **row}
                rows.append(full)
    return pd.DataFrame(rows)

# Source XML: prefer first entry in zf/xml_files, else fallback to example file
if 'zf' in globals() and 'xml_files' in globals() and xml_files:
    xml_text_src = zf.read(xml_files[0]).decode('utf-8-sig', errors='replace')
else:
    example_path = Path('notebooks/ActivationPriceXML.txt')
    if not example_path.exists():
        example_path = Path('../notebooks/ActivationPriceXML.txt')
    xml_text_src = example_path.read_text(encoding='utf-8')

points_full_df = parse_balancing_xml_to_df(xml_text_src)
print(points_full_df.shape)
display(points_full_df.head())


(716, 35)


,doc.mRID,doc.revisionNumber,doc.type,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.start,doc.end,...,ts.out_Domain.mRID,ts.contract_MarketAgreement.type,ts.currency_Unit.name,ts.price_Measure_Unit.name,ts.curveType,per.start,per.end,per.resolution,pt.position,pt.price.amount
0,d6c14f9c2cea473494634d78c262877d,1,A44,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-12T10:05:41Z,2025-08-31T22:00Z,2025-09-30T22:00Z,...,10YHU-MAVIR----U,A01,EUR,MWH,A03,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,1,88.15
1,d6c14f9c2cea473494634d78c262877d,1,A44,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-12T10:05:41Z,2025-08-31T22:00Z,2025-09-30T22:00Z,...,10YHU-MAVIR----U,A01,EUR,MWH,A03,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,2,87.34
2,d6c14f9c2cea473494634d78c262877d,1,A44,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-12T10:05:41Z,2025-08-31T22:00Z,2025-09-30T22:00Z,...,10YHU-MAVIR----U,A01,EUR,MWH,A03,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,3,86.33
3,d6c14f9c2cea473494634d78c262877d,1,A44,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-12T10:05:41Z,2025-08-31T22:00Z,2025-09-30T22:00Z,...,10YHU-MAVIR----U,A01,EUR,MWH,A03,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,4,85.36
4,d6c14f9c2cea473494634d78c262877d,1,A44,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-11-12T10:05:41Z,2025-08-31T22:00Z,2025-09-30T22:00Z,...,10YHU-MAVIR----U,A01,EUR,MWH,A03,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,5,87.05


In [6]:
points_full_df.columns

Index(['doc.mRID', 'doc.revisionNumber', 'doc.type',
       'doc.sender_MarketParticipant.mRID',
       'doc.sender_MarketParticipant.marketRole.type',
       'doc.receiver_MarketParticipant.mRID',
       'doc.receiver_MarketParticipant.marketRole.type', 'doc.createdDateTime',
       'doc.start', 'doc.end', 'doc.auction.type', 'doc.businessType',
       'doc.in_Domain.mRID', 'doc.out_Domain.mRID',
       'doc.contract_MarketAgreement.type', 'doc.currency_Unit.name',
       'doc.price_Measure_Unit.name', 'doc.curveType', 'doc.resolution',
       'doc.position', 'doc.price.amount', 'ts.mRID', 'ts.auction.type',
       'ts.businessType', 'ts.in_Domain.mRID', 'ts.out_Domain.mRID',
       'ts.contract_MarketAgreement.type', 'ts.currency_Unit.name',
       'ts.price_Measure_Unit.name', 'ts.curveType', 'per.start', 'per.end',
       'per.resolution', 'pt.position', 'pt.price.amount'],
      dtype='object')

In [7]:
points_full_df = points_full_df[["per.start", "per.end", "per.resolution", "pt.position", "pt.price.amount"]]

In [8]:
points_full_df

,per.start,per.end,per.resolution,pt.position,pt.price.amount
0,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,1,88.15
1,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,2,87.34
2,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,3,86.33
3,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,4,85.36
4,2025-08-31T22:00Z,2025-09-01T22:00Z,PT60M,5,87.05
...,...,...,...,...,...
711,2025-09-29T22:00Z,2025-09-30T22:00Z,PT60M,20,370.11
712,2025-09-29T22:00Z,2025-09-30T22:00Z,PT60M,21,192.89
713,2025-09-29T22:00Z,2025-09-30T22:00Z,PT60M,22,141.91
714,2025-09-29T22:00Z,2025-09-30T22:00Z,PT60M,23,124.01


In [ ]:

# parse UTC 'Z' strings
start_utc = pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
end_utc   = pd.to_datetime(points_full_df['per.end'],   utc=True, errors='coerce')

# DST-aware Hungary time (Europe/Budapest)
points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Europe/Budapest')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Europe/Budapest')

points_full_df['per.start_dt'] = start_utc.dt.tz_convert('Etc/GMT-2')
points_full_df['per.end_dt']   = end_utc.dt.tz_convert('Etc/GMT-2')

points_full_df.head()

KeyboardInterrupt: 

: 

In [ ]:
# Start of the day-period in Europe/Budapest
start_dt = (
    pd.to_datetime(points_full_df['per.start'], utc=True, errors='coerce')
      .dt.tz_convert('Europe/Budapest')
)

# Parse ISO 8601 resolution (PT15M/PT30M/PT60M/PT1H...) -> minutes
res_ex = points_full_df['per.resolution'].str.upper().str.extract(r'PT(\d+)([HM])')
mins = pd.to_numeric(res_ex[0], errors='coerce')
mins = np.where(res_ex[1].eq('H'), mins * 60, mins)
mins = pd.to_numeric(mins, errors='coerce')

# Positions are 1-based; compute offset and exact time
pos = pd.to_numeric(points_full_df['pt.position'], errors='coerce')
offset_min = (pos - 1) * mins

points_full_df['time_dt'] = start_dt + pd.to_timedelta(offset_min, unit='m')
points_full_df['time_end_dt'] = points_full_df['time_dt'] + pd.to_timedelta(mins, unit='m')

points_full_df[['per.start','per.resolution','pt.position','time_dt','time_end_dt']].head()

,per.start,per.resolution,pt.position,time_dt,time_end_dt
0,2025-08-31T22:00Z,PT60M,1,2025-09-01 00:00:00+02:00,2025-09-01 01:00:00+02:00
1,2025-08-31T22:00Z,PT60M,2,2025-09-01 01:00:00+02:00,2025-09-01 02:00:00+02:00
2,2025-08-31T22:00Z,PT60M,3,2025-09-01 02:00:00+02:00,2025-09-01 03:00:00+02:00
3,2025-08-31T22:00Z,PT60M,4,2025-09-01 03:00:00+02:00,2025-09-01 04:00:00+02:00
4,2025-08-31T22:00Z,PT60M,5,2025-09-01 04:00:00+02:00,2025-09-01 05:00:00+02:00


In [ ]:
points_full_df = points_full_df[['per.start_dt', 'per.end_dt' ,'time_dt','time_end_dt','per.resolution','pt.position', "pt.price.amount"]]

## Check for Missing Hourly Values

Verify that all hours are present in the dataset without gaps


In [ ]:
# Check for missing hourly values in day-ahead prices

# Get min and max datetime
min_dt = points_full_df['time_dt'].min()
max_dt = points_full_df['time_dt'].max()

print("=" * 80)
print("DAY-AHEAD PRICE DATA - COMPLETENESS CHECK")
print("=" * 80)
print(f"\nActual data:")
print(f"  Records: {len(points_full_df)}")
print(f"  Date range: {min_dt} to {max_dt}")
print(f"  Resolution: {points_full_df['per.resolution'].unique()}")

# Create complete hourly range
complete_hours = pd.date_range(
    start=min_dt.floor('h'),
    end=max_dt.ceil('h'),
    freq='h',
    tz='Europe/Budapest'
)

print(f"\nExpected hourly range:")
print(f"  Start: {complete_hours[0]}")
print(f"  End: {complete_hours[-1]}")
print(f"  Total hours: {len(complete_hours)}")

# Round time_dt to hour for comparison
points_full_df['time_dt_hour'] = points_full_df['time_dt'].dt.floor('h')

# Get unique hours in dataset
actual_hours = set(points_full_df['time_dt_hour'].unique())
expected_hours = set(complete_hours)

# Find missing hours
missing_hours = expected_hours - actual_hours

print(f"\n" + "=" * 80)
if len(missing_hours) == 0:
    print("✓ NO MISSING HOURS - Dataset is complete!")
else:
    print(f"✗ MISSING {len(missing_hours)} HOURS:")
    missing_sorted = sorted(list(missing_hours))
    for missing in missing_sorted[:20]:  # Show first 20
        print(f"  - {missing}")
    if len(missing_sorted) > 20:
        print(f"  ... and {len(missing_sorted) - 20} more")

# Check for duplicate hours
duplicate_check = points_full_df.groupby('time_dt_hour').size()
duplicates = duplicate_check[duplicate_check > 1]

print(f"\n" + "=" * 80)
if len(duplicates) == 0:
    print("✓ NO DUPLICATE HOURS - Each hour appears exactly once")
else:
    print(f"✗ FOUND {len(duplicates)} HOURS WITH MULTIPLE ENTRIES:")
    for hour, count in duplicates.head(10).items():
        print(f"  - {hour}: {count} entries")
    if len(duplicates) > 10:
        print(f"  ... and {len(duplicates) - 10} more")

print("=" * 80)


DAY-AHEAD PRICE DATA - COMPLETENESS CHECK

Actual data:
  Records: 716
  Date range: 2025-09-01 00:00:00+02:00 to 2025-09-30 23:00:00+02:00
  Resolution: ['PT60M']

Expected hourly range:
  Start: 2025-09-01 00:00:00+02:00
  End: 2025-09-30 23:00:00+02:00
  Total hours: 720

✗ MISSING 4 HOURS:
  - 2025-09-13 13:00:00+02:00
  - 2025-09-13 14:00:00+02:00
  - 2025-09-18 14:00:00+02:00
  - 2025-09-19 14:00:00+02:00

✓ NO DUPLICATE HOURS - Each hour appears exactly once


C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_11192\1863597083.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_full_df['time_dt_hour'] = points_full_df['time_dt'].dt.floor('h')


In [ ]:
points_full_df

,per.start_dt,per.end_dt,time_dt,time_end_dt,per.resolution,pt.position,pt.price.amount,time_dt_hour
0,2025-09-01 00:00:00+02:00,2025-09-02 00:00:00+02:00,2025-09-01 00:00:00+02:00,2025-09-01 01:00:00+02:00,PT60M,1,88.15,2025-09-01 00:00:00+02:00
1,2025-09-01 00:00:00+02:00,2025-09-02 00:00:00+02:00,2025-09-01 01:00:00+02:00,2025-09-01 02:00:00+02:00,PT60M,2,87.34,2025-09-01 01:00:00+02:00
2,2025-09-01 00:00:00+02:00,2025-09-02 00:00:00+02:00,2025-09-01 02:00:00+02:00,2025-09-01 03:00:00+02:00,PT60M,3,86.33,2025-09-01 02:00:00+02:00
3,2025-09-01 00:00:00+02:00,2025-09-02 00:00:00+02:00,2025-09-01 03:00:00+02:00,2025-09-01 04:00:00+02:00,PT60M,4,85.36,2025-09-01 03:00:00+02:00
4,2025-09-01 00:00:00+02:00,2025-09-02 00:00:00+02:00,2025-09-01 04:00:00+02:00,2025-09-01 05:00:00+02:00,PT60M,5,87.05,2025-09-01 04:00:00+02:00
...,...,...,...,...,...,...,...,...
711,2025-09-30 00:00:00+02:00,2025-10-01 00:00:00+02:00,2025-09-30 19:00:00+02:00,2025-09-30 20:00:00+02:00,PT60M,20,370.11,2025-09-30 19:00:00+02:00
712,2025-09-30 00:00:00+02:00,2025-10-01 00:00:00+02:00,2025-09-30 20:00:00+02:00,2025-09-30 21:00:00+02:00,PT60M,21,192.89,2025-09-30 20:00:00+02:00
713,2025-09-30 00:00:00+02:00,2025-10-01 00:00:00+02:00,2025-09-30 21:00:00+02:00,2025-09-30 22:00:00+02:00,PT60M,22,141.91,2025-09-30 21:00:00+02:00
714,2025-09-30 00:00:00+02:00,2025-10-01 00:00:00+02:00,2025-09-30 22:00:00+02:00,2025-09-30 23:00:00+02:00,PT60M,23,124.01,2025-09-30 22:00:00+02:00
